In [1]:
!pip install ranx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 kB 2.5 MB/s eta 0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 42.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 6.5 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 30.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 3.8 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.7/240.7 kB 6.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 29.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 MB 28.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 2.2 MB/s eta 0:00:00eta 0:00:01


In [2]:
from tira.third_party_integrations import ensure_pyterrier_is_loaded, normalize_run, persist_and_normalize_run
from tira.rest_api_client import Client
import pyterrier as pt
import pandas as pd
from tqdm import tqdm
from statistics import mean
from ranx import fuse, Run

/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
ensure_pyterrier_is_loaded()
tira = Client()


PyTerrier 0.10.1 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [4]:
LONGEVAL_LAGS = {
    'lag1': {
        'tira_id': 'longeval-2023-01-20240423-training',
        'pt_dataset': pt.get_dataset('irds:ir-lab-padua-2024/longeval-2023-01-20240426-training'),
    },
    'lag6': {
        'tira_id': 'longeval-2023-06-20240418-training',
        'pt_dataset': pt.get_dataset('irds:ir-lab-padua-2024/longeval-2023-06-20240422-training'),
    },
    'lag8': {
        'tira_id': 'longeval-2023-08-20240418-training',
        'pt_dataset': pt.get_dataset('irds:ir-lab-padua-2024/longeval-2023-08-20240422-training'),
    }
}

In [5]:
def to_ranx_dict(run, qrels = None):
    ret = {}
    for _, row in run.iterrows():
        if row['qid'] not in ret:
            ret[row['qid']] = {}

        ret[row['qid']][row['docno']] = row['score']

    if qrels is not None:
        for key in qrels.keys():
            if not key in ret.keys():
                ret[key] = {}
    

    return Run(ret)

def from_ranx(run):
    ret = []

    for qid in run.keys():
        for docno in run[qid].keys():
            ret += [{
                'qid': qid,
                'docno': docno,
                'score': run[qid][docno]
            }]
    return pd.DataFrame(ret)


In [6]:
from ranx import fuse, optimize_fusion, Qrels

qrels = Qrels.from_ir_datasets("ir-lab-padua-2024/longeval-2023-01-20240426-training")

print("BM25")
df_3 = tira.pd.from_retriever_submission('ir-benchmarks/tira-ir-starter/BM25 Re-Rank (tira-ir-starter-pyterrier)', LONGEVAL_LAGS["lag1"]['tira_id'])
print("Sparse Cross-Encoder")
df_4 = tira.pd.from_retriever_submission('ir-benchmarks/fschlatt/sparse-cross-encoder-4-512', LONGEVAL_LAGS["lag1"]['tira_id'])
print("ColBERT")
df_5 = tira.pd.from_retriever_submission('ir-benchmarks/tira-ir-starter/ColBERT Re-Rank (tira-ir-starter-pyterrier)', LONGEVAL_LAGS["lag1"]['tira_id'])
print("RankZephyr")
df_6 = normalize_run(tira.pd.from_retriever_submission('workshop-on-open-web-search/fschlatt/rank-zephyr', LONGEVAL_LAGS["lag1"]['tira_id']), "rank-zephyr")

run3 = to_ranx_dict(df_3, qrels)
run4 = to_ranx_dict(df_4, qrels)
run5 = to_ranx_dict(df_5, qrels)
run6 = to_ranx_dict(df_6, qrels)

best_params = optimize_fusion(
    qrels=qrels,
    runs=[run3, run4, run5, run6],
    norm="min-max",
    method="wsum",
    metric="ndcg@10", 
    show_progress = True,
    step=0.1, # -> best_params = (0.1, 0.1, 0.1, 0.7)
    # step=0.05, # -> best_params = (0.10, 0.10, 0.10, 0.70)
)
print(best_params)

for lag in tqdm(LONGEVAL_LAGS.keys()):
    print("BM25")
    df_3 = tira.pd.from_retriever_submission('ir-benchmarks/tira-ir-starter/BM25 Re-Rank (tira-ir-starter-pyterrier)', LONGEVAL_LAGS[lag]['tira_id'])
    print("Sparse Cross-Encoder")
    df_4 = tira.pd.from_retriever_submission('ir-benchmarks/fschlatt/sparse-cross-encoder-4-512', LONGEVAL_LAGS[lag]['tira_id'])
    print("ColBERT")
    df_5 = tira.pd.from_retriever_submission('ir-benchmarks/tira-ir-starter/ColBERT Re-Rank (tira-ir-starter-pyterrier)', LONGEVAL_LAGS[lag]['tira_id'])
    print("RankZephyr")
    df_6 = normalize_run(tira.pd.from_retriever_submission('workshop-on-open-web-search/fschlatt/rank-zephyr', LONGEVAL_LAGS[lag]['tira_id']), "rank-zephyr")

    run3 = to_ranx_dict(df_3, qrels)
    run4 = to_ranx_dict(df_4, qrels)
    run5 = to_ranx_dict(df_5, qrels)
    run6 = to_ranx_dict(df_6, qrels)
    
    print("fuse")
    df = fuse(
        runs=[run3, run4, run5, run6],
        norm="min-max",
        method="wsum",   
        params=best_params,
    )
    df = from_ranx(df)
    system = "galapagos-tortoise-wsum"
    normalize_run(df, system).to_csv(f'{system}.{lag}', sep=" ", header=False, index=False)

BM25
Sparse Cross-Encoder
ColBERT
RankZephyr


/home/codespace/.python/current/lib/python3.10/site-packages/ranx/normalization/min_max_norm.py:41: NumbaTypeSafetyWarning: unsafe cast from uint64 to int64. Precision may be lost.
  normalized_run[i] = _min_max_norm(run[q_ids[i]])


/home/codespace/.python/current/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" 
for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

/home/codespace/.python/current/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" 
for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

/home/codespace/.python/current/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" 
for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

...

Download: 2.32MiB [00:00, 16.8MiB/s]


Download finished. Extract...
Extraction finished:  /home/codespace/.tira/extracted_runs/ir-benchmarks/longeval-2023-08-20240418-training/tira-ir-starter
RankZephyr
The download is derived from The LongEval Dataset under the "Qwant LongEval Attribution-NonCommercial-ShareAlike License". Hence, the download is also under this License. By using it, you agree to the terms of this license. Please find details at: https://lindat.mff.cuni.cz/repository/xmlui/page/Qwant_LongEval_BY-NC-SA_License


Download: 1.14MiB [00:00, 9.79MiB/s]


Download finished. Extract...
Extraction finished:  /home/codespace/.tira/extracted_runs/workshop-on-open-web-search/longeval-2023-08-20240418-training/fschlatt
fuse


100%|██████████| 3/3 [03:03<00:00, 61.15s/it]


In [7]:
system = "galapagos-tortoise-rank-zephyr"
for lag in tqdm(LONGEVAL_LAGS.keys()):
    normalize_run(tira.pd.from_retriever_submission('workshop-on-open-web-search/fschlatt/rank-zephyr', LONGEVAL_LAGS[lag]['tira_id']), system).to_csv(f'{system}.{lag}', sep=" ", header=False, index=False)

100%|██████████| 3/3 [00:01<00:00,  2.51it/s]
